In [1]:
# !pip install mlflow

In [15]:
# !npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.124s


In [29]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import numpy as np
import json
import mlflow

In [3]:
'''When creating new experiment'''
# experiment_id = mlflow.create_experiment(
#     "MNIST MLP classifier", 
#     artifact_location=Path.cwd().joinpath("./mlruns").as_uri(),
#     tags={"version":"v1", "priority":"P1"}
# )

'When creating new experiment'

In [4]:
mlflow.set_tracking_uri("sqlite:///mlruns/mlruns.db")

In [5]:
experiment = mlflow.set_experiment("MNIST MLP classifier")

In [6]:
(train_images,train_labels),(test_images,test_labels) = mnist.load_data()
print(len(train_images))

60000


In [7]:
optim = "rmsprop"
epochs = 5
batch_size = 128

In [8]:
model = keras.Sequential([layers.Dense(128,activation='relu'),
layers.Dense(10,activation='softmax')])
model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

2022-12-09 13:01:50.556947: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-09 13:01:50.557363: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-09 13:01:50.557406: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f0a27f9ea0cc): /proc/driver/nvidia/version does not exist
2022-12-09 13:01:50.558006: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000,28*28))
test_images = test_images.astype("float32") / 255

In [10]:
with mlflow.start_run(experiment_id=experiment.experiment_id, description="MLP model for classifying MNIST hand written digits", run_name=f"{optim}-e{epochs}-{batch_size}") as run:
  mlflow.keras.autolog(registered_model_name="MLP MNIST classifier")
  history = model.fit(train_images,train_labels,epochs = epochs, batch_size = batch_size)

/usr/local/lib/python3.8/dist-packages/mlflow/utils/autologging_utils/__init__.py:416: FutureWarning: Autologging support for keras >= 2.6.0 has been deprecated and will be removed in a future MLflow release. Use `mlflow.tensorflow.autolog()` instead.
  return _autolog(*args, **kwargs)
2022/12/09 13:01:56 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.


Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.3305 - accuracy: 0.9086
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.1583 - accuracy: 0.9543
Epoch 3/5
469/469 [==============================] - 1s 3ms/step - loss: 0.1134 - accuracy: 0.9674
Epoch 4/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0879 - accuracy: 0.9745
Epoch 5/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0721 - accuracy: 0.9786
INFO:tensorflow:Assets written to: /tmp/tmpyu0sx8pp/model/data/model/assets


Registered model 'MLP MNIST classifier' already exists. Creating a new version of this model...
2022/12/09 13:02:09 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: MLP MNIST classifier, version 8
Created version '8' of model 'MLP MNIST classifier'.


In [11]:
# get_ipython().system_raw("mlflow server --host 0.0.0.0 -p 8000 --backend-store-uri sqlite:///mlruns/mlruns.db --default-artifact-root ./mlruns &")

In [12]:
# get_ipython().system_raw('lt --port 8000 >> url.txt 2>&1 &')

In [13]:
# !rm url.txt

In [14]:
# !cat url.txt

In [15]:
test_img = np.expand_dims(test_images[0][:], axis=0)
test_labl = test_labels[0]
pred_labl = model.predict(test_img)

1/1 [==============================] - 0s 80ms/step


In [16]:
# print(f"Predicted error {test_labl[0] - pred_labl[0].argmax()}") 

In [17]:
# test_loss, test_accuracy = model.evaluate(test_img,test_labl)
# print('Test Loss: ', test_loss)
# print('Test Accuracy: ',test_accuracy*100)

In [18]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)
print('Test Loss: ', test_loss)
print('Test Accuracy: ',test_accuracy*100)

313/313 [==============================] - 0s 1ms/step - loss: 0.0864 - accuracy: 0.9742
Test Loss:  0.08635642379522324
Test Accuracy:  97.42000102996826


## Loading model for predictions

In [19]:
run.info.run_id

'c4d64dfca658436e86338c7bd25f84be'

In [20]:
keras_model = mlflow.keras.load_model(f"runs:/{run.info.run_id}" + "/model")

In [21]:
predictions = keras_model.predict(test_img)

1/1 [==============================] - 0s 39ms/step


In [22]:
predictions[0].argmax()

7

In [23]:
test_images[0].shape

(784,)

## Infering with served model

In [24]:
get_ipython().system_raw(f"mlflow models serve --model-uri mlruns/{experiment.experiment_id}/{run.info.run_id}/artifacts/model --no-conda &")

/usr/local/lib/python3.8/dist-packages/click/core.py:2309: FutureWarning: `--no-conda` is deprecated and will be removed in a future MLflow release. Use `--env-manager=local` instead.
  value = self.callback(ctx, self, value)
2022/12/09 13:02:38 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2022/12/09 13:02:38 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2022-12-09 13:02:38 +0000] [562] [INFO] Starting gunicorn 20.1.0
[2022-12-09 13:02:38 +0000] [562] [INFO] Listening at: http://127.0.0.1:5000 (562)
[2022-12-09 13:02:38 +0000] [562] [INFO] Using worker: sync
[2022-12-09 13:02:38 +0000] [564] [INFO] Booting worker with pid: 564
2022-12-09 13:02:39.791466: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-crit

In [25]:
import requests
import json
import numpy as np

In [26]:
json_data = {'data': np.expand_dims(test_images[0], axis=0).tolist()}

In [27]:
headers = {'Content-type': 'application/json'}
res = requests.post('http://127.0.0.1:5000/invocations', headers=headers, data=json.dumps(json_data))
print(res.text)

1/1 [==============================] - 0s 71ms/step
[{"0": 5.624719179309068e-08, "1": 1.4245428170411856e-09, "2": 0.0001184048960567452, "3": 0.0011782062938436866, "4": 1.6664601920624023e-09, "5": 1.4434417607844807e-06, "6": 1.701980778534562e-10, "7": 0.9986528158187866, "8": 2.7376670459489105e-06, "9": 4.6316399675561115e-05}]


In [31]:
res_json = json.loads(res.text)

In [39]:
max(res_json[0], key=res_json[0].get)

'7'